In [39]:
from itertools import product
from random import random, randint, seed, uniform, sample
import numpy as np
import math
from scipy import sparse
from copy import copy
from tqdm import tqdm
from functools import reduce

In [40]:
def make_set_covering_problem(num_points, num_sets, density, prob):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    np.random.seed(int(num_points * 435761 + num_sets + density))
    sets = sparse.lil_array((num_sets, num_points), dtype=bool).toarray()
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    initial_state = np.random.choice([True, False], size=(num_sets,), p=[prob, 1 - prob])
    return sets, initial_state

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [41]:
NUM_SETS = 1000
NUM_POINTS = 1000
DENSITY = 0.3
PROB = 0.02

problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY, PROB)
print(
    f'Problem shape: {problem.shape}',
    f'Initial state shape: {initial_state.shape}, Taken sets: {np.sum(initial_state)}',
    sep='\n',
)

Problem shape: (1000, 1000)
Initial state shape: (1000,), Taken sets: 10


In [42]:
def check_goal(problem, state):
    return np.all(
        reduce(
            np.logical_or,
            [problem[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )

In [43]:
assert check_goal(problem, np.full((NUM_SETS,), True)), "Problem not solvable"

In [44]:
def fitness(problem, state):
    goal = check_goal(problem, state)
    cost = np.sum(state)
    return goal, cost if not goal else -cost

In [45]:
fitness(problem, initial_state)

(False, 10)

In [46]:
def random_tweak(state):
    """
    Tweak a state randomly.

    Args:
        state: 1-D boolean ndarray.

    Returns:
        New state with a changed boolean value.
    """
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state


def tweak_by_index(state, index):
    """
    Tweak a state changing the value in position index.

    Args:
        state: 1-D boolean ndarray;
        index: int value indicating the boolean value to change.

    Returns:
        New state with a changed boolean value according
        to the value of index.
    """
    new_state = copy(state)
    new_state[index] = not new_state[index]
    return new_state

In [47]:
def RMHC(problem, state, fitness, max_it, max_give_up):
    """
    Random-Mutation Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations (int);
        max_give_up: maximum number of evaluations before giving up (int).

    Returns:
        Possible state solution to the problem.
    """
    changes = 0
    evals_giveup = 0
    for evals in tqdm(range(max_it)):
        new_state = random_tweak(state)
        if fitness(problem, new_state) > fitness(problem, state):
            state = new_state
            changes += 1
            evals_giveup = 0
        else:
            evals_giveup += 1
            if evals_giveup == max_give_up:
                print('Maximum number of evaluations without improvement reached.')
                break

    evals += 1
    if evals == max_it:
        print('Maximum number of iterations reached.')
    print(f'Terminated after {changes} changes.', f'Number of evaluations: {evals}.', sep='\n')

    goal, cost = fitness(problem, state)

    print(f'Goal reached? {"Yes" if goal else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [48]:
_ = RMHC(problem, initial_state, fitness, 10_000, 2_000)

 28%|██▊       | 2822/10000 [00:00<00:01, 4556.78it/s]

Maximum number of evaluations without improvement reached.
Terminated after 13 changes.
Number of evaluations: 2823.
Goal reached? Yes
State cost: 15


In [49]:
def SAHC(problem, state, fitness, max_it):
    """
    Steepest-Ascent Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    evals = 0
    changes = 0
    for it in tqdm(range(max_it)):
        succ = tweak_by_index(state, 0)
        for index in range(1, NUM_SETS):
            new_state = tweak_by_index(state, index)
            evals += 1
            if fitness(problem, new_state) > fitness(problem, succ):
                succ = new_state
        evals += 1
        if fitness(problem, succ) > fitness(problem, state):
            state = succ
            changes += 1
        else:
            break

    it += 1
    if it == max_it:
        print('Maximum number of iterations reached.')
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = fitness(problem, state)

    print(f'Goal reached? {"Yes" if goal else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [50]:
_ = SAHC(problem, initial_state, fitness, 1_000)

  1%|          | 7/1000 [00:01<04:00,  4.13it/s]

Terminated after 8 iterations.
Terminated after 7 changes.
Number of evaluations: 8000.
Goal reached? Yes
State cost: 13


In [51]:
def SAHCwReplacement(problem, state, fitness, n_neighbors, max_it):
    """
    Steepest-Ascent Hill Climber implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        n_neighbors: number of desired tweaks to try;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    evals = 0
    changes = 0
    for it in tqdm(range(max_it)):
        index = randint(0, NUM_SETS)
        succ = tweak_by_index(state, index)
        for index in set(sample(range(NUM_SETS), n_neighbors)) - {index}:
            new_state = tweak_by_index(state, index)
            evals += 1
            if fitness(problem, new_state) > fitness(problem, succ):
                succ = new_state
        evals += 1
        if fitness(problem, succ) > fitness(problem, state):
            state = succ
            changes += 1
        else:
            break

    it += 1
    if it == max_it:
        print('Maximum number of iterations reached.')
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = fitness(problem, state)

    print(f'Goal reached? {"Yes" if goal else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [52]:
_ = SAHCwReplacement(problem, initial_state, fitness, 100, 1_000)

  1%|          | 6/1000 [00:00<00:25, 38.96it/s]

Terminated after 7 iterations.
Terminated after 6 changes.
Number of evaluations: 705.
Goal reached? Yes
State cost: 16


In [53]:
def simulated_annealing(problem, state, fitness, max_it):
    """
    Simulated Annealing implementation.

    Args:
        problem: 2-D boolean ndarray;
        state: 1-D boolean ndarray (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations (int).

    Returns:
        Possible state solution to the problem.
    """
    evals = 0
    changes = 0
    for it in tqdm(range(max_it)):
        T = 1 - ((it + 1) / max_it)
        if T == 0:
            break
        new_state = random_tweak(state)
        evals += 1
        new_value, old_value = fitness(problem, new_state), fitness(problem, state)
        if new_value > old_value:
            state = new_state
            changes += 1
        else:
            if uniform(0, 1) < math.exp((new_value[1] - old_value[1]) / T):
                state = new_state
                changes += 1

    it += 1
    print(
        f'Terminated after {it} iterations.',
        f'Terminated after {changes} changes.',
        f'Number of evaluations: {evals}.',
        sep='\n',
    )

    goal, cost = fitness(problem, state)

    print(f'Goal reached? {"Yes" if goal else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [54]:
_ = simulated_annealing(problem, initial_state, fitness, 5_900)

100%|█████████▉| 5899/5900 [00:01<00:00, 2950.48it/s]

Terminated after 5900 iterations.
Terminated after 1466 changes.
Number of evaluations: 5899.
Goal reached? Yes
State cost: 18
